## Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module

In [2]:
from keys import key_iex

In [3]:
sp500=pd.read_csv('sp_500_stocks.csv')
sp500

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [10]:
cols=['Ticker', 'Price','Price-to-Earnings Ratio', 'Number Of Shares to Buy']
stock_param=pd.DataFrame(columns=cols )

In [5]:
from create_batches import create_batches

In [6]:
res=list(create_batches(sp500['Ticker'],100))

In [7]:
batch_req=[]
for i in res:
    batch_req.append(','.join(i))
print(batch_req)

['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA', 'CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD', 'GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX

In [11]:
#updated way
for symbol in batch_req:
    ot=requests.get(f"https://api.iex.cloud/v1/data/core/quote/{symbol}?token={key_iex}").json()
    for cs in ot:
        stock_param.loc[len(stock_param.index)] = [cs['symbol'], 
                                                    cs['latestPrice'], 
                                                    cs['peRatio'], 
                                                    'N/A']

In [12]:
stock_param

,Ticker,Price,Price-to-Earnings Ratio,Number Of Shares to Buy
0,A,130.56,31.16,N/A
1,AAL,13.60,5.07,N/A
2,AAP,61.52,19.72,N/A
3,AAPL,181.18,29.56,N/A
4,ABBV,162.14,44.42,N/A
...,...,...,...,...
500,YUM,128.34,24.4,N/A
501,ZBH,119.98,53.09,N/A
502,ZBRA,252.69,28.17,N/A
503,ZION,44.05,4.7,N/A


Now we need to sort this values in increasing ordere with respect to price to earing ratio but we will need to drop the ones with negative peratio because this means that company is in loss which is not a good place to invest money

In [15]:
stock_param.sort_values('Price-to-Earnings Ratio',inplace=True)
stock_param=stock_param[stock_param['Price-to-Earnings Ratio']>0]

,Ticker,Price,Price-to-Earnings Ratio,Number Of Shares to Buy
71,BRK.B,365.59,0.01,N/A
192,FRC,3.51,0.43,N/A
137,DISH,5.77,2.9,N/A
159,EMR,95.47,4.17,N/A
468,VLO,129.00,4.39,N/A
...,...,...,...,...
273,KSU,293.59,279.61,N/A
493,WYNN,95.65,324.68,N/A
238,IFF,79.97,971.69,N/A
31,AMD,138.58,1080.12,N/A


In [16]:
best50=stock_param[:50]
best50

,Ticker,Price,Price-to-Earnings Ratio,Number Of Shares to Buy
71,BRK.B,365.59,0.01,N/A
192,FRC,3.51,0.43,N/A
137,DISH,5.77,2.9,N/A
159,EMR,95.47,4.17,N/A
468,VLO,129.00,4.39,N/A
503,ZION,44.05,4.7,N/A
23,ALB,135.83,4.81,N/A
454,UAL,41.76,4.84,N/A
203,GM,35.99,5.07,N/A
1,AAL,13.60,5.07,N/A


In [18]:
best50.reset_index(inplace=True)

,level_0,index,Ticker,Price,Price-to-Earnings Ratio,Number Of Shares to Buy
0,0,71,BRK.B,365.59,0.01,N/A
1,1,192,FRC,3.51,0.43,N/A
2,2,137,DISH,5.77,2.9,N/A
3,3,159,EMR,95.47,4.17,N/A
4,4,468,VLO,129.00,4.39,N/A
5,5,503,ZION,44.05,4.7,N/A
6,6,23,ALB,135.83,4.81,N/A
7,7,454,UAL,41.76,4.84,N/A
8,8,203,GM,35.99,5.07,N/A
9,9,1,AAL,13.60,5.07,N/A


In [22]:
best50.drop('index',axis=1,inplace=True)
best50

C:\Users\sjmad\AppData\Local\Temp\ipykernel_20576\1900234139.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best50.drop(['index','level_0'],axis=1,inplace=True)


,Ticker,Price,Price-to-Earnings Ratio,Number Of Shares to Buy
0,BRK.B,365.59,0.01,N/A
1,FRC,3.51,0.43,N/A
2,DISH,5.77,2.9,N/A
3,EMR,95.47,4.17,N/A
4,VLO,129.00,4.39,N/A
5,ZION,44.05,4.7,N/A
6,ALB,135.83,4.81,N/A
7,UAL,41.76,4.84,N/A
8,GM,35.99,5.07,N/A
9,AAL,13.60,5.07,N/A


Now lets invest 10000000 in the stocks accordingly

In [23]:
su=20*len(best50.index)-best50['Price-to-Earnings Ratio'].sum()
print(su)

633.69


In [25]:
best50['Price-to-Earnings Ratio']=pd.to_numeric(best50['Price-to-Earnings Ratio'])
best50.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Ticker                   50 non-null     object 
 1   Price                    50 non-null     float64
 2   Price-to-Earnings Ratio  50 non-null     float64
 3   Number Of Shares to Buy  50 non-null     object 
dtypes: float64(2), object(2)
memory usage: 1.7+ KB


C:\Users\sjmad\AppData\Local\Temp\ipykernel_20576\2069741467.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best50['Price-to-Earnings Ratio']=pd.to_numeric(best50['Price-to-Earnings Ratio'])


In [29]:
for i in range (0,50):
    best50.loc[i,'Number Of Shares to Buy']=math.floor(((20-best50.loc[i,'Price-to-Earnings Ratio']/su)*10000000)/ best50.loc[i,'Price'])
best50

,Ticker,Price,Price-to-Earnings Ratio,Number Of Shares to Buy
0,BRK.B,365.59,0.01,547060
1,FRC,3.51,0.43,56978123
2,DISH,5.77,2.90,34654113
3,EMR,95.47,4.17,2094209
4,VLO,129.00,4.39,1549850
5,ZION,44.05,4.70,4538611
6,ALB,135.83,4.81,1471869
7,UAL,41.76,4.84,4787443
8,GM,35.99,5.07,5554876
9,AAL,13.60,5.07,14699999


C:\Users\sjmad\AppData\Local\Temp\ipykernel_20576\1450900138.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best50.drop('Number of Shares to Buy',axis=1,inplace=True)


,Ticker,Price,Price-to-Earnings Ratio,Number Of Shares to Buy
0,BRK.B,365.59,0.01,N/A
1,FRC,3.51,0.43,N/A
2,DISH,5.77,2.90,N/A
3,EMR,95.47,4.17,N/A
4,VLO,129.00,4.39,N/A
5,ZION,44.05,4.70,N/A
6,ALB,135.83,4.81,N/A
7,UAL,41.76,4.84,N/A
8,GM,35.99,5.07,N/A
9,AAL,13.60,5.07,N/A
